In [13]:
import re
import os
import logging
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

tqdm.pandas()
logging.basicConfig(level=logging.DEBUG)

In [2]:
SENTIMENT_COLUMN = 'target'
TEXT_COLUMN = 'text'
RAW_TEXT_COLUMN = 'raw_text'

In [3]:
def read_files(path):
    train_pos_files = os.listdir(path)
    raw_text = []
    for filename in train_pos_files:
        fb = open(path+filename,'r')
        raw_text.append(fb.readline())
        fb.close()  
    return raw_text

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
def normalize(words):
    word_list = []
    for word in words:
        """Remove non-ASCII characters from list of tokenized words"""
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        """Convert all characters to lowercase from list of tokenized words"""
        new_word = new_word.lower()
        """Remove punctuation except for "?" and "!" from list of tokenized words"""
        if contraction_mapping.__contains__(new_word):
            new_word = contraction_mapping[new_word]
        new_word = re.sub(r'[^\w\s!?]', '', new_word)
        """Lemmatize verbs in list of tokenized words"""
        lemmatizer = WordNetLemmatizer()
        new_word = lemmatizer.lemmatize(new_word, pos='v')
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        if new_word.isdigit():
            new_word = 'digit'
        if new_word != '':
            word_list.append(new_word)
    return word_list

In [5]:
# read all train data
train_pos = read_files('./aclImdb/train/pos/')
train_neg = read_files('./aclImdb/train/neg/')
raw_train_text = train_pos + train_neg
train_label = [1] * len(train_pos) + [0] * len(train_neg)
raw_train_text, train_label = shuffle(raw_train_text, train_label, random_state=0)

# read all test data
test_pos = read_files('./aclImdb/test/pos/')
test_neg = read_files('./aclImdb/test/neg/')
raw_test_text = test_pos + test_neg
test_label = [1] * len(test_pos) + [0] * len(test_neg)
raw_test_text, test_label = shuffle(raw_test_text, test_label, random_state=0)

raw_corpus_text = raw_train_text + raw_test_text

In [6]:
from nltk.stem import WordNetLemmatizer
from tokenizer import tokenize
import re, unicodedata, inflect
train_text = []
for each in raw_train_text:
    tmp = each.replace('-', ' ').replace('<br />',"")
    train_text.append("".join(word + ' ' for word in normalize([word.txt for word in tokenize(tmp) if word.txt != None])).strip())
test_text = []
for each in raw_test_text:
    tmp = each.replace('-', ' ').replace('<br />',"")
    test_text.append("".join(word + ' ' for word in normalize([word.txt for word in tokenize(tmp) if word.txt != None])).strip())
processed_corpus_text = train_text + test_text

In [7]:
MAX_SEQUENCE_LEN = 512
MAX_FEATURES = 100000

In [8]:
"""
Load data
"""

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

train = pd.DataFrame(list(zip(raw_train_text,train_text, train_label)), columns =[RAW_TEXT_COLUMN,TEXT_COLUMN, SENTIMENT_COLUMN])
test = pd.DataFrame(list(zip(raw_test_text,test_text, test_label)), columns =[RAW_TEXT_COLUMN,TEXT_COLUMN, SENTIMENT_COLUMN])
train_label = train[SENTIMENT_COLUMN]

Using TensorFlow backend.


In [15]:
"""
Fit tokenizer
"""

logging.info("Fitting Tokenizer")

tokenizer = Tokenizer(num_words=MAX_FEATURES, lower=True)
# Updates internal vocabulary based on a list of texts
tokenizer.fit_on_texts(list(train[TEXT_COLUMN]) + list(test[TEXT_COLUMN]))

logging.info("Beginning process train text")
train_text = tokenizer.texts_to_sequences(list(train[TEXT_COLUMN]))
train_text = pad_sequences(train_text, maxlen=MAX_SEQUENCE_LEN)

logging.info("Beginning process test text")
test_text = tokenizer.texts_to_sequences(list(test[TEXT_COLUMN]))
test_text = pad_sequences(test_text, maxlen=MAX_SEQUENCE_LEN)

INFO:root:Fitting Tokenizer
INFO:root:Beginning process train text
INFO:root:Beginning process test text


In [16]:
word_index = tokenizer.word_index
del tokenizer
gc.collect()

0

In [17]:
"""
Get word Embedding
"""

EMB_PATH = "skipgram_model_IMDB.vec"
EMBEDDING_DIM = 300

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(embed_dir=EMB_PATH):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embed_dir)))
    return embedding_index

def build_matrix(word_index, embeddings_index):
    embedding_matrix = np.zeros((len(word_index) + 1,EMBEDDING_DIM))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embeddings_index[word]
        except:
            embedding_matrix[i] = embeddings_index["unknown"]
    return embedding_matrix

In [18]:
embeddings_index = load_embeddings()
embedding_matrix = build_matrix(word_index, embeddings_index)
del embeddings_index
gc.collect()

76832it [00:13, 5848.09it/s]


0

In [25]:
"""
Model builder
"""

from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPooling1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

def build_model(verbose = False, compile = True):
    sequence_input = Input(shape=(MAX_SEQUENCE_LEN,), dtype='int32')
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LEN,
                                trainable=False)
    
    x = embedding_layer(sequence_input)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    preds = Dense(1, activation='sigmoid')(x)
    
    model = Model(sequence_input, preds)
    if verbose:
        model.summary()
    if compile:
        model.compile(loss='binary_crossentropy',
                      optimizer='Adam',
                      metrics=['acc'])
    return model

In [26]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 512)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 512, 300)          30914400  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512, 128)          186880    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total para

In [30]:
"""
Training Session
"""
from keras import backend as K
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint


BATCH_SIZE = 1024
NUM_EPOCHS = 100
FOLDER_NUM = 2

splits = list(KFold(n_splits=FOLDER_NUM).split(train_text, train_label))

test_preds = np.zeros((test_text.shape[0]))
folders = [i for i in range(FOLDER_NUM)]

for folder in folders:
    K.clear_session()
    train_index, validate_index = splits[folder]
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    model = build_model()
    model.fit(x = train_text[train_index],
              y = train_label[train_index],
              batch_size = BATCH_SIZE,
              epochs = NUM_EPOCHS,
              validation_data = (train_text[validate_index],
                                 train_label[validate_index]),
              callbacks = [es])
    test_preds += model.predict(test_text)[:, 0]
test_preds /= FOLDER_NUM

Train on 12500 samples, validate on 12500 samples
Epoch 1/100
12500/12500 [==============================] - 161s 13ms/step - loss: 0.6708 - acc: 0.6132 - val_loss: 0.6318 - val_acc: 0.6881
Epoch 2/100
12500/12500 [==============================] - 154s 12ms/step - loss: 0.5742 - acc: 0.7252 - val_loss: 0.4704 - val_acc: 0.7914
Epoch 3/100
12500/12500 [==============================] - 159s 13ms/step - loss: 0.4364 - acc: 0.8040 - val_loss: 0.3971 - val_acc: 0.8322
Epoch 4/100
12500/12500 [==============================] - 155s 12ms/step - loss: 0.4005 - acc: 0.8189 - val_loss: 0.3989 - val_acc: 0.8135
Epoch 5/100
12500/12500 [==============================] - 150s 12ms/step - loss: 0.3728 - acc: 0.8372 - val_loss: 0.3405 - val_acc: 0.8550
Epoch 6/100
12500/12500 [==============================] - 149s 12ms/step - loss: 0.3504 - acc: 0.8482 - val_loss: 0.3278 - val_acc: 0.8611
Epoch 7/100
12500/12500 [==============================] - 156s 12ms/step - loss: 0.3256 - acc: 0.8615 - val_l

In [31]:
test_predect = [1 if each > 0.5 else 0 for each in test_preds ]
print("==========Model Evalucation=========")
print(classification_report(test_label, test_predect))
print("============Test Accuracy===========")
print("acc:", balanced_accuracy_score(test_label, test_predect))

==========Model Evalucation=========
              precision    recall  f1-score   support

           0       0.90      0.86      0.88     12500
           1       0.87      0.90      0.88     12500

   micro avg       0.88      0.88      0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

============Test Accuracy===========
acc: 0.88124
